# SkyLink Navigator - Main ATC Agent with Tools

This notebook implements the main SkyLink Navigator agent that has access to 4 tools and uses Claude Sonnet for final response generation.

## Workflow Overview:
```
START → ATC_MAIN_AGENT → Claude Sonnet Response → END
           ↓
   ┌─────────────────┐
   │     TOOLS       │
   │  • GeoTracker   │
   │  • Scheduler    │
   │  • Weather      │
   │  • CommsAgent   │
   └─────────────────┘
```

In [0]:
%pip install -r requirements.txt

In [0]:
dbutils.library.restartPython()

In [0]:
from databricks_langchain import ChatDatabricks
from databricks.sdk import WorkspaceClient
import os

w = WorkspaceClient()

os.environ["DATABRICKS_HOST"] = w.config.host
os.environ["DATABRICKS_TOKEN"] = w.tokens.create(comment="for model serving", lifetime_seconds=1200).token_value

llm = ChatDatabricks(endpoint="databricks-llama-4-maverick")

In [0]:
from pydantic import BaseModel

class CommsAnalysisResponse(BaseModel):
    commsAnalysis: str

In [0]:
from langchain.agents import initialize_agent, Tool
from langchain.tools import tool
from langchain.agents.agent_types import AgentType
from langgraph.graph import StateGraph, END, START
from langgraph.prebuilt import ToolNode
import sqlite3
import os

@tool
def query_flight_schedule(sql: str) -> str:
    """Run a SQL query on the flight_status.db file."""
    try:
        conn = sqlite3.connect("flight_status.db")
        cursor = conn.cursor()
        cursor.execute(sql)
        rows = cursor.fetchall()
        col_names = [description[0] for description in cursor.description]
        conn.close()
        return "\n".join([str(dict(zip(col_names, row))) for row in rows]) or "No results."
    except Exception as e:
        return f"SQL Error: {str(e)}"


@tool
def query_geotracking(sql: str) -> str:
    """Run a SQL query on the geotracking.db file."""
    try:
        conn = sqlite3.connect("geo_tracking.db")
        cursor = conn.cursor()
        cursor.execute(sql)
        rows = cursor.fetchall()
        col_names = [description[0] for description in cursor.description]
        conn.close()
        return "\n".join([str(dict(zip(col_names, row))) for row in rows]) or "No results."
    except Exception as e:
        return f"SQL Error: {str(e)}"
    


@tool
def query_weather(sql: str) -> str:
    """Run a SQL query on the weather.db file."""
    try:
        conn = sqlite3.connect("weather.db")
        cursor = conn.cursor()
        cursor.execute(sql)
        rows = cursor.fetchall()
        columns = [desc[0] for desc in cursor.description]
        conn.close()
        return "\n".join([str(dict(zip(columns, row))) for row in rows]) or "No results."
    except Exception as e:
        return f"SQL Error in Weather Tool: {e}"
    
@tool
def comms_agent(message: str) -> CommsAnalysisResponse:
    """
    Analyze pilot communication and return LLM analysis.
    Returns simple JSON: {"commsAnalysis": "analysis text"}
    """
    from databricks_langchain import ChatDatabricks
    
    # Initialize Databricks LLM
    llm = ChatDatabricks(
        endpoint="databricks-meta-llama-3-3-70b-instruct",
    )
    
    # Simple analysis prompt
    analysis_prompt = f"""
    You are an Air Traffic Controller analyzing pilot communication. 
    Provide a brief analysis of this message including intent, urgency, and any key information extracted.

    PILOT MESSAGE: "{message}"

    Provide a concise analysis in 1-2 sentences:
    """
    
    llm = llm.with_structured_output(CommsAnalysisResponse)
    return llm.invoke(analysis_prompt)
            
    #     except Exception as e:
    #         # Fallback analysis
    #         fallback_text = _simple_fallback_analysis(message)
    #         result = CommsAnalysisResponse(commsAnalysis=fallback_text)
    #         return result.model_dump_json()
            
    # except Exception as e:
    #     # Error fallback
    #     error_result = CommsAnalysisResponse(
    #         commsAnalysis=f"Communication analysis error: {str(e)}. Message received: {message[:50]}..."
    #     )
    #     return error_result.model_dump_json()
    
# def _simple_fallback_analysis(message: str) -> str:
#     """Simple rule-based fallback analysis"""
#     msg = message.lower()
    
#     # Extract callsign
#     callsign = "Aircraft"
#     for word in message.split():
#         if len(word) >= 3 and word.replace('-', '').isalnum():
#             callsign = word.upper()
#             break
    
#     # Quick analysis
#     if "mayday" in msg or "emergency" in msg:
#         return f"EMERGENCY detected from {callsign}. Immediate priority handling required."
#     elif "clearance" in msg:
#         return f"{callsign} requesting clearance. Standard priority, prepare clearance delivery."
#     elif "weather" in msg:
#         return f"{callsign} requesting weather information. Routine request."
#     elif "taxi" in msg:
#         return f"{callsign} requesting taxi clearance. Ground movement coordination needed."
#     elif "traffic" in msg:
#         return f"{callsign} requesting traffic advisory. Check surrounding aircraft positions."
#     else:
#         return f"General communication from {callsign}. Standard acknowledgment required."

In [0]:
tools = [
    Tool(
        name="ScheduleTrackerTool",
        func=query_flight_schedule,
        description="Use this tool to query scheduled flights and detect conflicts, delays, or tight arrival overlaps. Accepts SQL input."
    ),
    Tool(
        name="GeoTrackerTool",
        func=query_geotracking,
        description="Use this tool to query geospatial data about flight phases and deviations from expected routes."
    ),
    Tool(
        name="WeatherTrackerTool",
        func=query_weather,
        description="Use this tool to query weather_by_flight table to get wind, visibility, storm/fog info, and help determine flight risk."
    )
    ,
    Tool(
        name="CommsAnalysisTool",
        func=comms_agent,
        description="Use this tool to analyze pilot communication and provide a brief analysis of intent, urgency, and key information extracted."
    )
]

In [0]:
from typing import Dict, Any, List, Optional, Annotated
from typing_extensions import TypedDict
from langchain.schema import BaseMessage, HumanMessage, AIMessage, SystemMessage
from langgraph.checkpoint.base import BaseCheckpointSaver

class ATCState(TypedDict):
    """
    State management for tool-based ATC workflow
    """
    # Input and identification
    messages: Annotated[List[BaseMessage], lambda x, y: x + y if isinstance(y, list) else x + [y]]

In [0]:
def receive_pilot_message(state: ATCState) -> Dict[str, Any]:
    """
    Main ATC Agent with Databricks LLM-driven tool selection and response generation
    """
    print("🤖 LLM Tool Selector: Analyzing pilot request...")
    

    pass

In [0]:
class SkyLinkNavigator:
    """
    Main ATC Agent with Databricks LLM-driven tool selection and response generation
    """
    
    def __init__(self):
        self.graph = None
        self.tools = tools
        self.tool_node = ToolNode(self.tools)
        
        # Initialize Databricks LLM for intelligent tool selection and response generation
        self.llm = ChatDatabricks(
            endpoint="databricks-llama-4-maverick",
        )
        
        self._build_atc_workflow()
    
    def _build_atc_workflow(self):
        """
        Build LLM-driven ATC workflow: Input → LLM Tool Selection → Tool Execution → LLM Response → End
        """
        workflow = StateGraph(ATCState)

        # Define workflow nodes
        workflow.add_node("receive_pilot_message", receive_pilot_message)


        # Define workflow edges
        workflow.add_edge(START, "receive_pilot_message")
        workflow.add_edge("receive_pilot_message", END)

        self.graph = workflow.compile()
        
        # # Main workflow nodes
        # workflow.add_node("llm_tool_selector", self._llm_tool_selection)
        # workflow.add_node("execute_tools", self._execute_selected_tools)
        # workflow.add_node("llm_response_generator", self._llm_generate_response)
        
        # # Set entry point
        # workflow.set_entry_point("llm_tool_selector")
        
        # # LLM-driven workflow
        # workflow.add_edge("llm_tool_selector", "execute_tools")
        # workflow.add_edge("execute_tools", "llm_response_generator")
        # workflow.add_edge("llm_response_generator", END)
        
        # self.graph = workflow.compile()
        # print("✅ LLM-Driven ATC Workflow compiled successfully")

In [0]:
navigator = SkyLinkNavigator()

In [0]:
from IPython.display import Image, display

display(Image(navigator.graph.get_graph().draw_mermaid_png()))

In [0]:
# {"messages": [{"role": "user", "content": "book a stay at McKittrick hotel"}

navigator.graph.invoke("Hi There!")

In [0]:
def stream_graph_updates(user_input: str):
    for event in navigator.graph.stream({"messages": [{"role": "user", "content": user_input}]}):
        for value in event.values():
            print("Assistant:", value["messages"][-1].content)

while True:
    user_input = input("User: ")
    if user_input.lower() in ["quit", "exit", "q"]:
        print("Goodbye!")
        break

    stream_graph_updates(user_input)